In [1]:
# ::::::::::::Mark:::::::::::
# I implement most of my work in Kaggle Notebook and run on Kaggle platform,
# so it may not function properly in other platforms.





# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/compettioon/train_label.npz
/kaggle/input/compettioon/example_submission.csv
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/455feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/1640feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/862feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/2003feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/573feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/1981feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/412feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/549feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/71feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/873feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/1387feature.npz
/kaggle/input/compettioon/feature_npz_test/feature_npz_test/2444feature.npz
/kagg

In [2]:
# process the label data
import numpy as np

# load training labels
label_path = '/kaggle/input/compettioon/train_label.npz'
label = np.load(label_path)

print(list(label.keys())) # labels
train_labels = label['labels']


print(type(train_labels)) #nd array
print(train_labels.shape) #(11886,)

['labels']
<class 'numpy.ndarray'>
(11886,)


In [3]:
# process the feature data
import numpy as np
# load feature
feature_path = '/kaggle/input/feature-separate-train/feature_npz_train/'

feature_list = []

for i in range(11886):
    feature = np.load(feature_path + str(i) + "feature.npz")
#     print(list(feature.keys()))
    f = feature['arr_0'].reshape((299,40)) # 299*40
#     print(f.shape)
    feature_list.append(f)


feature_combined = np.stack(feature_list, axis=0)
print(feature_combined.shape) # (11886, 299, 40)

(11886, 299, 40)


In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        # make sure that feature is a 2 dimensional array [height, width]
        feature_tensor = torch.tensor(feature, dtype=torch.float32).unsqueeze(0)  # add dimemsion to [1, height, width]
        label_tensor = torch.tensor(label, dtype=torch.long)
        return feature_tensor, label_tensor




from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature_combined, train_labels, test_size=0.3, random_state=8)


dataset_train = MyDataset(X_train, y_train)
dataset_test = MyDataset(X_test, y_test)

In [5]:
# process the prediction data
import numpy as np
# load feature
feature_path_pred = '/kaggle/input/feature-separate-test/feature_npz_test/'

feature_list_pred = []

for i in range(2447):
    feature_pred = np.load(feature_path_pred + str(i) + "feature.npz")
#     print(list(feature.keys()))
    f1 = feature_pred['arr_0'].reshape((299,40)) # 299*40
#     print(f1.shape)
    feature_list_pred.append(f1)


feature_combined_pred = np.stack(feature_list_pred, axis=0)
print(feature_combined_pred.shape) # (2447, 299, 40)
dataset_pred = MyDataset(feature_combined_pred,np.empty((2447, 0)))



(2447, 299, 40)


In [6]:
# use Dataloader to load data
batch_size = 32
n_models = 20
dropout_probability=0.2
train_loaders = [torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True) for _ in range(n_models)]
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
pred_loader = torch.utils.data.DataLoader(dataset_pred, batch_size=batch_size, shuffle=False)

In [7]:
import torch
import torch.nn as nn
from torchvision import models
import numpy as np

class ResNetWithDropout(nn.Module):
    def __init__(self, num_classes=4, dropout_probability=0.2, input_channels=1):
        super(ResNetWithDropout, self).__init__()
        self.base_model = models.resnet34(pretrained=False)
        self.base_model.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Dropout(dropout_probability),
            nn.Linear(num_features, num_classes)
        )

    def forward(self, x):
        return self.base_model(x)

class EnsembleModel(nn.Module):
    def __init__(self, n_models, num_classes, dropout_probability, input_channels):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList([
            ResNetWithDropout(num_classes=num_classes, dropout_probability=dropout_probability, input_channels=input_channels)
            for _ in range(n_models)
        ])
    
    def forward(self, x):
        # use average probability method, suitable for classification problem
        outputs = [model(x) for model in self.models]
        avg_output = torch.mean(torch.stack(outputs), dim=0)
        return avg_output

input_channels=1
num_classes=4



In [8]:
# Dense Net

# import torch
# import torch.nn as nn
# from torchvision import models
# import numpy as np

# class CustomDenseNet(nn.Module):
#     def __init__(self, num_classes=4):
#         super(CustomDenseNet, self).__init__()
#         self.densenet = models.densenet121(pretrained=False)
#         self.densenet.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
#         num_features = self.densenet.classifier.in_features
#         self.densenet.classifier = nn.Linear(num_features, num_classes)

#     def forward(self, x):
#         features = self.densenet.features(x)
#         out = nn.ReLU()(features)
#         out = nn.AdaptiveAvgPool2d((1, 1))(out)
#         out = torch.flatten(out, 1)
#         out = self.densenet.classifier(out)
#         return out



# class EnsembleModel(nn.Module):
#     def __init__(self, n_models, num_classes, input_channels):
#         super(EnsembleModel, self).__init__()
#         self.models = nn.ModuleList([
#             CustomDenseNet(num_classes)
#             for _ in range(n_models)
#         ])
    
#     def forward(self, x):
#         outputs = [model(x) for model in self.models]
#         avg_output = torch.mean(torch.stack(outputs), dim=0)
#         return avg_output

# input_channels=1
# num_classes=4

In [14]:
# set model
num_epochs=10
# ensemble_model = EnsembleModel(n_models, num_classes, input_channels)  # this is for dense net
ensemble_model = EnsembleModel(n_models, num_classes, dropout_probability, input_channels) # this is for resnet

In [15]:
import torch.optim as optim
from tqdm import tqdm

# set optimizer and loss function
optimizers = [optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5) for model in ensemble_model.models]
for i in range(len(optimizers)):
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizers[i], T_0=num_epochs*len(train_loaders[i])//2)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model.to(device)

# train model
def train_model(num_epochs, train_loaders, test_loader, ensemble_model, optimizers):
    for epoch in range(num_epochs):
        ensemble_model.train()  # Set the ensemble model to training mode
        for sub_model in ensemble_model.models:
            sub_model.train()  # Set each sub-model to training mode

        total_loss = 0
        
        # Iterate over each model, optimizer, and corresponding train loader
        for model, optimizer, loader in zip(ensemble_model.models, optimizers, train_loaders):
            for inputs, labels in tqdm(loader):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            lr_scheduler.step()
        avg_loss = total_loss / sum(len(loader) for loader in train_loaders)
        print(f'Epoch {epoch+1}, Loss: {avg_loss}')
        evaluate_model(test_loader, ensemble_model)

# evaluate on validation set
def evaluate_model(test_loader, ensemble_model):
    ensemble_model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = [model(inputs) for model in ensemble_model.models]
            outputs = torch.mean(torch.stack(outputs), dim=0)  # Average the outputs
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on validation set: {100 * correct / total}%')

# Train the ensemble model
train_model(num_epochs, train_loaders=train_loaders, test_loader=test_loader, ensemble_model=ensemble_model, optimizers=optimizers)


100%|██████████| 260/260 [00:13<00:00, 19.08it/s]


Epoch 1, Loss: 0.7362347286710372


100%|██████████| 112/112 [00:43<00:00,  2.59it/s]


Accuracy on test set: 79.08020190689848%


100%|██████████| 260/260 [00:13<00:00, 19.12it/s]


...

Accuracy on test set: 86.06281547952888%


100%|██████████| 260/260 [00:13<00:00, 19.02it/s]


Epoch 10, Loss: 0.2719167326254627


100%|██████████| 112/112 [00:43<00:00,  2.58it/s]

Accuracy on test set: 86.84800897363993%


In [26]:
train_model(1, train_loaders=train_loaders, test_loader=test_loader, ensemble_model=ensemble_model, optimizers=optimizers)

100%|██████████| 260/260 [00:13<00:00, 19.04it/s]


Epoch 1, Loss: 0.20819420953890166


100%|██████████| 112/112 [00:43<00:00,  2.58it/s]

Accuracy on test set: 87.01626472237801%


In [27]:
# empty the output folder
!rm -rf /kaggle/working/*

In [28]:
# predict on the test set
import pandas as pd
predictions = []
def predict_output(model,predict_loader,predictions):

    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        for inputs, labels in pred_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = [model(inputs) for model in ensemble_model.models]
            outputs = torch.mean(torch.stack(outputs), dim=0)  # Average the outputs
            _, predicted = torch.max(outputs.data, 1)   
            if predicted.requires_grad:
                predicted = predicted.detach() # detach gradient computing
            if predicted.is_cuda:
                predicted = predicted.cpu()

            predicted = predicted.numpy()
#             print(predicted)
            predictions.append(predicted)
#             print(len(predicted))
#             print(len(predictions))
            concatenated_pred = np.concatenate(predictions, axis=0)

    return concatenated_pred

concatenated_pred = predict_output(ensemble_model,pred_loader,predictions)


# print(concatenated_pred)
submission_df = pd.DataFrame({
    "id": range(len(concatenated_pred)),  # create a range of numbers from 0 to the number of predictions
    "category": concatenated_pred  # fill in the prediction results
})

# save DataFrame as CSV 
submission_file_path = '/kaggle/working/ensemble.csv'  # path to save the submission file
submission_df.to_csv(submission_file_path, index=False)  # make sure not to save the index column

print(f"CSV file saved to {submission_file_path}")

CSV file saved to /kaggle/working/ensemble.csv


In [25]:
# save the model as PTH file

cnt = 0
for model in ensemble_model.models:
    torch.save(model, '/kaggle/working/modelll_'+ str(cnt)+'.pth')
    cnt += 1

In [ ]:
# import zipfile
# import os

# # Path to the directory containing the files you want to download
# folder_path = '/kaggle/working/'

# # Path to the zip file you want to create (this will save in the working directory)
# zip_path = '/kaggle/working/modelss.zip'

# # Creating a zip file
# with zipfile.ZipFile(zip_path, 'w') as zipf:
#     for file in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, file)
#         zipf.write(file_path, arcname=file)  # arcname strips the provided path and saves only the file

# print("Zip file created!")